In [3]:
# Import necessary packages
import pandas as pd
import os
import glob

# 1) Find columns in GPS data

In [4]:
# Load in only column names of GPS data in memory (cannot load full 9GB dataset)
df = pd.read_csv(f'GPS_Data/caleb_AOI_full_traces.tsv', sep='\t', nrows=0)
print(df.columns.tolist())

['Country', 'Hashed_ID', 'lat', 'lon', 'Unix_Time']


# 2) Order tsv file by user id and unix time in Git Bash

Order TSV by user ID and unix time based on column locations found above and save ordered GPS data:

(head -n 1 "Input_File_Location" && \
 tail -n +2 "Input_File_Location" | \
 sort -t $'\t' -k2,2 -k5,5n --parallel=10) > "Output_File_Location"

Row 2 is hashed ID and row 5 is unix time.

File contains 118,431,864 rows (found using: wc -l file_location.tsv in Git Bash)

The file contains 14,179 unique Hashed_IDs within the dataset

Dataset ranges from July 20th 2020 00:00:00 NZT to December 1st 2020 11:59:58 NZT

These values were found via: cut f(row num) file_location.tsv | sourt -u -- parallel=10 | wc -l

# 3) Split dataset into ~1 million row chunks for memory management

In [5]:
# Define filepath to tsv and output directory
input_file = "GPS_Data/AOI_sorted_traces.tsv"
output_folder = "GPS_Data/Chunks"

# Create output directory
os.makedirs(output_folder, exist_ok=True)

chunksize = 1_000_000 # Number of rows per chunk
chunk_num = 1 # Chunk counter to be updated
current_chunk = []  # will hold rows for the current output chunk
carryover_rows = []  # rows that must be carried over to next chunk

# Read TSV in chunks
for chunk in pd.read_csv(input_file, sep='\t', chunksize=chunksize, dtype=str):
    
    # Add carryover rows from previous chunk to current chunk
    if carryover_rows:
        chunk = pd.concat([pd.DataFrame(carryover_rows), chunk], ignore_index=True)
        carryover_rows = []

    # Determine rows where Hashed_ID changes and save locations
    hashed_ids = chunk['Hashed_ID']
    change_points = hashed_ids.ne(hashed_ids.shift()).to_numpy().nonzero()[0]

    start_idx = 0
    # Loop over change points
    for idx in change_points[1:]:
        # Find if distance from current hashed_id change row to start row is larger than desired chunksize
        if idx - start_idx >= chunksize:
            # Slice data from start_idx to idx
            chunk_to_save = chunk.iloc[start_idx:idx]

            # Save slice containing all hashed_id data
            output_path = os.path.join(output_folder, f"GPS_Data_Chunk_{chunk_num}.tsv")
            chunk_to_save.to_csv(output_path, sep='\t', index=False)
            print(f"Saved {output_path} with {len(chunk_to_save)} rows")
            
            chunk_num += 1 #update chunk number counter
            start_idx = idx # update start_idx value

    # Any remaining rows go into carryover for next iteration
    carryover_rows = chunk.iloc[start_idx:].to_dict('records')

# Save any remaining rows
if carryover_rows:
    chunk_df = pd.DataFrame(carryover_rows)
    output_path = os.path.join(output_folder, f"GPS_Data_Chunk_{chunk_num}.tsv")
    chunk_df.to_csv(output_path, sep='\t', index=False)
    print(f"Saved {output_path} with {len(chunk_df)} rows")


Saved GPS_Data/Chunks\GPS_Data_Chunk_1.tsv with 1030944 rows
Saved GPS_Data/Chunks\GPS_Data_Chunk_2.tsv with 1024932 rows
Saved GPS_Data/Chunks\GPS_Data_Chunk_3.tsv with 1026649 rows
Saved GPS_Data/Chunks\GPS_Data_Chunk_4.tsv with 1006113 rows
Saved GPS_Data/Chunks\GPS_Data_Chunk_5.tsv with 1006825 rows
Saved GPS_Data/Chunks\GPS_Data_Chunk_6.tsv with 1011662 rows
Saved GPS_Data/Chunks\GPS_Data_Chunk_7.tsv with 1033503 rows
Saved GPS_Data/Chunks\GPS_Data_Chunk_8.tsv with 1002410 rows
Saved GPS_Data/Chunks\GPS_Data_Chunk_9.tsv with 1067184 rows
Saved GPS_Data/Chunks\GPS_Data_Chunk_10.tsv with 1029449 rows
Saved GPS_Data/Chunks\GPS_Data_Chunk_11.tsv with 1004333 rows
Saved GPS_Data/Chunks\GPS_Data_Chunk_12.tsv with 1010463 rows
Saved GPS_Data/Chunks\GPS_Data_Chunk_13.tsv with 1001427 rows
Saved GPS_Data/Chunks\GPS_Data_Chunk_14.tsv with 1020890 rows
Saved GPS_Data/Chunks\GPS_Data_Chunk_15.tsv with 1061502 rows
Saved GPS_Data/Chunks\GPS_Data_Chunk_16.tsv with 1012161 rows
Saved GPS_Data/Ch

# 4) Get aggregated stats by hashed ID

In [8]:
# Define file paths
chunk_folder = "GPS_Data/Chunks"
output_file = "GPS_Data/Hashed_ID_Summary.csv"

# List all chunk files
chunk_files = sorted(glob.glob(os.path.join(chunk_folder, "GPS_Data_Chunk_*.tsv")))

# Initialize a list to collect summariesT
all_summaries = []

for chunk_file in chunk_files:
    print(f"Processing {chunk_file} ...")
    # Extract chunk number
    digits = [char for char in chunk_file if char.isdigit()]
    chunk_num = int(''.join(digits))
    
    # Load the chunk and group by hashed id
    df = pd.read_csv(chunk_file, sep='\t')
    grouped = df.groupby('Hashed_ID')
    
    # Find key info by hashed ID
    summary = grouped.agg(
    num_rows=('Hashed_ID', 'size'),
    min_unix_time=('Unix_Time', 'min'),
    max_unix_time=('Unix_Time', 'max')
    ).reset_index()

    summary['TSV'] = chunk_num
    
    # Append summary variable to the all_summaries list
    all_summaries.append(summary)

# Convert all cummaries into a dataframe
final_summary = pd.concat(all_summaries, ignore_index=True)

# Save to CSV
final_summary.to_csv(output_file, index=False)
print(f"Final summary saved to {output_file}")

Processing GPS_Data/Chunks\GPS_Data_Chunk_1.tsv ...
Processing GPS_Data/Chunks\GPS_Data_Chunk_10.tsv ...
Processing GPS_Data/Chunks\GPS_Data_Chunk_100.tsv ...
Processing GPS_Data/Chunks\GPS_Data_Chunk_101.tsv ...
Processing GPS_Data/Chunks\GPS_Data_Chunk_102.tsv ...
Processing GPS_Data/Chunks\GPS_Data_Chunk_103.tsv ...
Processing GPS_Data/Chunks\GPS_Data_Chunk_104.tsv ...
Processing GPS_Data/Chunks\GPS_Data_Chunk_105.tsv ...
Processing GPS_Data/Chunks\GPS_Data_Chunk_106.tsv ...
Processing GPS_Data/Chunks\GPS_Data_Chunk_107.tsv ...
Processing GPS_Data/Chunks\GPS_Data_Chunk_108.tsv ...
Processing GPS_Data/Chunks\GPS_Data_Chunk_109.tsv ...
Processing GPS_Data/Chunks\GPS_Data_Chunk_11.tsv ...
Processing GPS_Data/Chunks\GPS_Data_Chunk_110.tsv ...
Processing GPS_Data/Chunks\GPS_Data_Chunk_111.tsv ...
Processing GPS_Data/Chunks\GPS_Data_Chunk_112.tsv ...
Processing GPS_Data/Chunks\GPS_Data_Chunk_113.tsv ...
Processing GPS_Data/Chunks\GPS_Data_Chunk_114.tsv ...
Processing GPS_Data/Chunks\GPS_D